In [1]:
"The ECG shows sinus rhythm with peripheral low voltage; no heart axis data available; diagnostic classification: normal; no signs of infarction detected"

'The ECG shows sinus rhythm with peripheral low voltage; no heart axis data available; diagnostic classification: normal; no signs of infarction detected'

In [ ]:
!pip install beyondllm
!pip install llama-index-embeddings-huggingface
!pip install llama-index-embeddings-fastembed
!pip install ollama


In [1]:
from beyondllm import source, embeddings, retrieve, llms, generator

data = source.fit(
    path="/Users/sanjaibalajee/Development/GitHub/Multimodal_ECG/RAG_Pipeline/The-ECG-Made-Easy-9th-Edition.pdf",
    dtype="pdf",
    chunk_size=1024,
    chunk_overlap=0)



[nltk_data] Downloading package punkt_tab to /Users/sanjaibalajee/.pye
[nltk_data]     nv/versions/3.10.15/lib/python3.10/site-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
data

[TextNode(id_='ad4847d9-9542-4777-98bd-6707673e3d6a', embedding=None, metadata={'page_label': '1', 'file_name': 'The-ECG-Made-Easy-9th-Edition.pdf', 'file_path': '/Users/sanjaibalajee/Development/GitHub/Multimodal_ECG/RAG_Pipeline/The-ECG-Made-Easy-9th-Edition.pdf', 'file_type': 'application/pdf', 'file_size': 16895043, 'creation_date': '2024-10-02', 'last_modified_date': '2024-10-01'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='e11bc5b0-3951-452b-96f8-f06678a62c41', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '1', 'file_name': 'The-ECG-Made-Easy-9th-Edition.pdf', 'file_path': '/Users/sanjaibalajee/Development/GitHub/Multimodal_ECG/RAG_Pipeline/The-ECG-Made-Easy-9th-Edition.pdf', 'fil

In [2]:


from beyondllm.embeddings import HuggingFaceEmbeddings

embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# from beyondllm.embeddings import FastEmbedEmbeddings

# embed_model = FastEmbedEmbeddings()

# # text_embeddings = embed_model.embed_text("Huggingface models are awesome!")

# model_name='BAAI/bge-small-en-v1.5'
# embed_model = embeddings.HuggingFaceEmbeddings(


# model_name=model_name
# )


In [ ]:
#check working 
embed_model.embed_text("Normal ECG")

[0.028855744749307632,
 0.07159720361232758,
 -0.02061503194272518,
 0.047697585076093674,
 -0.027432763949036598,
 -0.07257131487131119,
 0.020319167524576187,
 0.02474553883075714,
 0.0262675192207098,
 -0.08815952390432358,
 0.04389065131545067,
 -0.03002195991575718,
 -0.044121891260147095,
 -0.008688647300004959,
 -0.07501541823148727,
 -0.03653042018413544,
 0.022637348622083664,
 -0.0902666375041008,
 -0.04474509134888649,
 0.06627611815929413,
 -0.08606132864952087,
 0.07326744496822357,
 0.007339550647884607,
 0.01903323270380497,
 -0.020344698801636696,
 0.00011070263280998915,
 0.0816955715417862,
 -0.0024330776650458574,
 0.07983484119176865,
 -0.019046122208237648,
 0.006565547548234463,
 -0.04256290942430496,
 0.09750489145517349,
 0.02943340130150318,
 -0.027276063337922096,
 0.07427996397018433,
 0.055524516850709915,
 -0.05960824340581894,
 -0.07401248812675476,
 0.016595201566815376,
 0.06935817003250122,
 0.012194563634693623,
 0.014962814748287201,
 -0.0430093742907

In [3]:
retriever = retrieve.auto_retriever(
data=data,
embed_model=embed_model,
type="cross-rerank",
mode="OR",
top_k=2)
if embed_model is not None:
    print("Embedding model loaded successfully")
if retriever is not None:
    print("Retriever loaded successfully")

Embedding model loaded successfully
Retriever loaded successfully


In [4]:
from beyondllm.llms import OllamaModel
    
llm = OllamaModel(model="llama3.1")

In [5]:
# prompt = """
# Based on the text, just improve the the input with relevant information regarding the ECG. The response should not be more than 2 sentences.DO NOT ADD ANYTHING ELSE TO THE RESPONSE.
# IF YOU DO NOT HAVE RELAVANT INFORMATION, JUST RETURN THE ORIGINAL INPUT. DO NOT TOO MUCH INFORMATION. FOR EXAMPLE IF THE INPUT IS "THE PATIENT SHOWS NORMAL ECG", 
# THE RESPONSE SHOULD ADD A LITTLE BIT MORE INFORMATION LIKE "THE PATIENT SHOWS NORMAL ECG WITH A HEART RATE OF X BPM" with this you can add other informaztion relavant as well.
# """


prompt="""enhance the input text with concise, relevant ecg details in no more than two sentences. add information only if it meaningfully clarifies the ecg, such as specific heart rate or rhythm details. if no additional information is relevant, return the input as-is without modifications.
"""

In [6]:
pipeline = generator.Generate(
question="The ECG shows sinus rhythm with peripheral low voltage; no heart axis data available; diagnostic classification: normal; no signs of infarction detected",
retriever=retriever,
system_prompt=prompt,
llm=llm)

In [7]:
print(pipeline.call())

Here is the modified input text with concise, relevant ECG details in two sentences:

"Sinus rhythm with normal PR interval (120 ms) and QRS complex duration (80 ms), as well as isoelectric ST segment and normal T waves in all leads except for inversion in lead VR."


In [8]:
print(pipeline.get_rag_triad_evals())

Executing RAG Triad Evaluations...
Context relevancy Score: 8.5
This response meets the evaluation threshold. It demonstrates strong comprehension and coherence.
Answer relevancy Score: 9.0
This response meets the evaluation threshold. It demonstrates strong comprehension and coherence.
Groundness score: 9.0
This response meets the evaluation threshold. It demonstrates strong comprehension and coherence.


In [23]:
from nltk.translate.bleu_score import sentence_bleu

reference = "Normal sinus rhythm with peripheral low voltage; no heart axis data available; diagnostic classification: normal; no infarction detected"
generated = " NORM; heart axis: no heart axis data;  No infarction detected"

# Tokenize the reference and generated sentences
reference_tokens = reference.split()
generated_tokens = generated.split()

# Calculate BLEU score
score = sentence_bleu([reference_tokens], generated_tokens)
print("BLEU Score:", score)
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

smoothing_function = SmoothingFunction().method4
score = sentence_bleu([reference_tokens], generated_tokens, smoothing_function=smoothing_function)
print("BLEU Score with smoothing:", score)



BLEU Score: 2.0849239249333461e-78
BLEU Score with smoothing: 0.07269985042101909


In [36]:
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score
from bert_score import score

# Texts
reference = "The ECG shows sinus rhythm with peripheral low voltage; no heart axis data available; diagnostic classification: normal; no signs of infarction detected"
generated = "Sinus rhythm with normal PR interval (120 ms) and QRS complex duration (80 ms), as well as isoelectric ST segment and normal T waves in all leads except for inversion in lead VR"

# Tokenize for METEOR
reference_tokens = reference.split()
generated_tokens = generated.split()

# Calculate ROUGE scores
rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = rouge_scorer_obj.score(reference, generated)

# Print ROUGE scores with evaluation
print("ROUGE Scores:")
print(f"ROUGE-1: {rouge_scores['rouge1'].fmeasure} - {'Good' if rouge_scores['rouge1'].fmeasure > 0.5 else 'Needs Improvement'}")
print(f"ROUGE-2: {rouge_scores['rouge2'].fmeasure} - {'Good' if rouge_scores['rouge2'].fmeasure > 0.3 else 'Needs Improvement'}")
print(f"ROUGE-L: {rouge_scores['rougeL'].fmeasure} - {'Good' if rouge_scores['rougeL'].fmeasure > 0.5 else 'Needs Improvement'}")

# # Calculate METEOR score
# meteor = meteor_score([' '.join(reference_tokens)], ' '.join(generated_tokens))
# print("\nMETEOR Score:", meteor, "- Good" if meteor > 0.5 else "Needs Improvement")

# Calculate BERTScore
P, R, F1 = score([generated], [reference], lang="en")
bert_f1 = F1.mean().item()
print("\nBERTScore F1:", bert_f1, "- Good" if bert_f1 > 0.85 else "Moderate" if bert_f1 > 0.7 else "Needs Improvement")


ROUGE Scores:
ROUGE-1: 0.14545454545454545 - Needs Improvement
ROUGE-2: 0.07547169811320754 - Needs Improvement
ROUGE-L: 0.14545454545454545 - Needs Improvement


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



BERTScore F1: 0.8202335834503174 Moderate


In [14]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Test sentences
reference_text = "This is a test sentence."
generated_text = "This is a test sentence."

# Tokenize both reference and generated text to ensure they match exactly
reference_tokens = reference_text.split()
generated_tokens = generated_text.split()

# Calculate BLEU without smoothing
bleu_score_no_smoothing = sentence_bleu([reference_tokens], generated_tokens)

# Calculate BLEU with smoothing (for comparison)
smoothing_function = SmoothingFunction().method4
bleu_score_with_smoothing = sentence_bleu([reference_tokens], generated_tokens, smoothing_function=smoothing_function)

print(f"BLEU Score without smoothing: {bleu_score_no_smoothing}")
print(f"BLEU Score with smoothing: {bleu_score_with_smoothing}")


BLEU Score without smoothing: 1.0
BLEU Score with smoothing: 1.0
